In [2]:
library(keras)

In [2]:
# path to train and test directories
train_path <- "fruits/train/"
test_path <- "fruits/test/"

In [3]:
class_label <- list.dirs(path = train_path, full.names = FALSE, recursive = TRUE)[-1]

In [4]:
class_label

[1] "Apricot"            "Avocado"            "Banana"            
 [4] "Cactus fruit"       "Cherry Wax Red"     "Chestnut"          
 [7] "Dates"              "Guava"              "Kiwi"              
[10] "Lemon"              "Lychee"             "Mango"             
[13] "Orange"             "Papaya"             "Peach"             
[16] "Pear"               "Physalis with Husk" "Pineapple"         
[19] "Pomegranate"        "Raspberry"          "Strawberry"        
[22] "Tomato Cherry Red"  "Walnut"

In [5]:
length(class_label)

[1] 23

In [4]:
img_width = 20
img_height = 20
img_size = c(img_width,img_height)

In [7]:
## read from train path
train_data <- flow_images_from_directory(directory = train_path,
                                         target_size = img_size,
                                         color_mode = "rgb",
                                         class_mode = "categorical",
                                         classes = class_label,
                                         batch_size = 20)

In [24]:
# read from test path
test_data <- flow_images_from_directory(directory = test_path,
                                        target_size = img_size,
                                        color_mode = "rgb",
                                        class_mode = "categorical",
                                        classes = class_label,
                                        batch_size = 20)

In [25]:
class(train_data)

[1] "python.builtin.iterator"                                       
[2] "keras_preprocessing.image.directory_iterator.DirectoryIterator"
[3] "keras_preprocessing.image.iterator.BatchFromFilesMixin"        
[4] "keras_preprocessing.image.iterator.Iterator"                   
[5] "keras.utils.data_utils.Sequence"                               
[6] "python.builtin.object"

In [10]:
print(paste("Number of images in train and test is",train_data$n,"and ",test_data$n,"repectively"))

[1] "Number of images in train and test is 11397 and  3829 repectively"


In [11]:
table(factor(train_data$classes))


  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 
492 427 490 490 492 450 490 490 466 492 490 490 479 492 492 492 492 490 492 490 
 20  21  22 
492 492 735 

In [12]:
table(factor(test_data$classes))



  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 
164 143 166 166 164 153 166 166 156 164 166 166 160 164 164 164 164 166 164 166 
 20  21  22 
164 164 249 

Notice that class labels are numeric let us look at the mapping of class label and class label names

In [13]:
test_data$class_indices

$Apricot
[1] 0

$Avocado
[1] 1

$Banana
[1] 2

$`Cactus fruit`
[1] 3

$`Cherry Wax Red`
[1] 4

$Chestnut
[1] 5

$Dates
[1] 6

$Guava
[1] 7

$Kiwi
[1] 8

$Lemon
[1] 9

$Lychee
[1] 10

$Mango
[1] 11

$Orange
[1] 12

$Papaya
[1] 13

$Peach
[1] 14

$Pear
[1] 15

$`Physalis with Husk`
[1] 16

$Pineapple
[1] 17

$Pomegranate
[1] 18

$Raspberry
[1] 19

$Strawberry
[1] 20

$`Tomato Cherry Red`
[1] 21

$Walnut
[1] 22

In [14]:
train_data$class_indices

$Apricot
[1] 0

$Avocado
[1] 1

$Banana
[1] 2

$`Cactus fruit`
[1] 3

$`Cherry Wax Red`
[1] 4

$Chestnut
[1] 5

$Dates
[1] 6

$Guava
[1] 7

$Kiwi
[1] 8

$Lemon
[1] 9

$Lychee
[1] 10

$Mango
[1] 11

$Orange
[1] 12

$Papaya
[1] 13

$Peach
[1] 14

$Pear
[1] 15

$`Physalis with Husk`
[1] 16

$Pineapple
[1] 17

$Pomegranate
[1] 18

$Raspberry
[1] 19

$Strawberry
[1] 20

$`Tomato Cherry Red`
[1] 21

$Walnut
[1] 22

In [15]:
train_data$image_shape

[[1]]
[1] 20

[[2]]
[1] 20

[[3]]
[1] 3

In [16]:
cnn_model_pool <- keras_model_sequential() %>% 
    layer_conv_2d(filters = 32, kernel_size = c(3,3), activation = 'relu',
                input_shape = c(img_width,img_height,3),padding = "same") %>% 
    layer_conv_2d(filters = 16, kernel_size = c(3,3), activation = 'relu',padding = "same") %>%
    layer_max_pooling_2d(pool_size = c(2,2)) %>%
    layer_flatten() %>% 
    layer_dense(units = 50, activation = 'relu') %>% 
    layer_dense(units = 23, activation = 'softmax')

In [17]:
cnn_model_pool %>% summary()

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
conv2d (Conv2D)                     (None, 20, 20, 32)              896         
________________________________________________________________________________
conv2d_1 (Conv2D)                   (None, 20, 20, 16)              4624        
________________________________________________________________________________
max_pooling2d (MaxPooling2D)        (None, 10, 10, 16)              0           
________________________________________________________________________________
flatten (Flatten)                   (None, 1600)                    0           
________________________________________________________________________________
dense (Dense)                       (None, 50)                      80050       
________________________________________________________________________________
dense_1 (Dense)             

In [18]:
# Compile model
cnn_model_pool %>% compile(
  loss = "categorical_crossentropy",
  optimizer = optimizer_rmsprop(lr = 0.0001,decay = 1e-6),
  metrics = c('accuracy')
)

In [19]:
# Train model
cnn_model_pool %>% fit_generator(generator = train_data,
                                 steps_per_epoch = 20,
                                 epochs = 5)

In [20]:
cnn_model_pool %>% save_model_hdf5("cnn_model_pool.h5")

# cnn_model <- load_model_hdf5("cnn_model_pool.h5")

In [21]:
scores <- cnn_model_pool %>% evaluate_generator(generator = test_data,steps = 20)

In [22]:
# Output metrics
paste('Test loss:', scores[[1]], '\n')
paste('Test accuracy:', scores[[2]], '\n')

[1] "Test loss: 2.9508512192228 \n"

[1] "Test accuracy: 0.789501190185547 \n"